<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Разделим-выборки" data-toc-modified-id="Разделим-выборки-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Разделим выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>SGDClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('Solarize_Light2')
import time

from IPython.display import display
import torch
import transformers
from tqdm import notebook
from tqdm import tqdm

import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer 


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier


from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

from sklearn.utils import shuffle


from IPython.display import Audio
wave = np.sin(2*np.pi*400*np.arange(10000*2)/10000)



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df_comments = pd.read_csv('/datasets/toxic_comments.csv')

# приведем текст к нижнему регистру
df_comments['text'] = df_comments['text'].str.lower()

df_comments.head()


,Unnamed: 0,text,toxic
0,0,explanation\nwhy the edits made under my usern...,0
1,1,d'aww! he matches this background colour i'm s...,0
2,2,"hey man, i'm really not trying to edit war. it...",0
3,3,"""\nmore\ni can't make any real suggestions on ...",0
4,4,"you, sir, are my hero. any chance you remember...",0


In [3]:
# проверка сбалансированности
df_comments['toxic'].value_counts() 


0    143106
1     16186
Name: toxic, dtype: int64

Наблюдается ярко выраженный дисбаланс классов.

In [4]:
corpus = list(df_comments['text'])

In [5]:
lemmatizer = WordNetLemmatizer()

#Создадим функцию по очистке данных
def clear_text(text):
    y=re.sub(r"[^'a-zA-Z ]", ' ', text) 
    k=" ".join(y.split())
    return k
 
#Создадим функцию которая будет использовать функцию по очистке слов и лемматизировать каждое слово

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)    



In [6]:
%%time

# Запустим преобразование текста

def lemmafunction(textc):
    k=[]
    for i in nltk.word_tokenize(textc):
        y=lemmatizer.lemmatize(i, get_wordnet_pos(i))
        k.append(y)
    return ' '.join(k) 

lemy=[]
for i in tqdm(range(len(corpus))):
    
    lemy.append(lemmafunction(clear_text(corpus[i])))
df_comments['lemm_text']=pd.Series(lemy, index=df_comments.index)

Audio(wave, rate=10000, autoplay=True)

100%|██████████| 159292/159292 [19:03<00:00, 139.27it/s]

CPU times: user 16min 55s, sys: 1min 44s, total: 18min 40s
Wall time: 19min 3s


Посмотрим, что получилось:

In [7]:
df_comments['text'][8]

"sorry if the word 'nonsense' was offensive to you. anyway, i'm not intending to write anything in the article(wow they would jump on me for vandalism), i'm merely requesting that it be more encyclopedic so one can use it for school as a reference. i have been to the selective breeding page but it's almost a stub. it points to 'animal breeding' which is a short messy article that gives you no info. there must be someone around with expertise in eugenics? 93.161.107.169"

In [8]:
df_comments['lemm_text'][8]

"sorry if the word 'nonsense ' be offensive to you anyway i 'm not intend to write anything in the article wow they would jump on me for vandalism i 'm merely request that it be more encyclopedic so one can use it for school a a reference i have be to the selective breeding page but it 's almost a stub it point to 'animal breeding ' which be a short messy article that give you no info there must be someone around with expertise in eugenics"

### Разделим выборки

In [9]:
#Разделим целевые значения и признаки
X=df_comments.drop('toxic',axis=1)
y=df_comments['toxic']


In [10]:
# разделим данные на тестовые и обучающие выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True,stratify=y)


In [11]:
#Проверим результат
train_sample=X_train.shape[0]/X.shape[0]
test_sample=X_test.shape[0]/X.shape[0]

print('Размер тренировочной выборки- {:.0%}'.format(train_sample))
print('Размер тестовой выборки - {:.0%}'.format(test_sample))

Размер тренировочной выборки- 90%
Размер тестовой выборки - 10%


Используем TF-IDF для обработки наших данных, а так же отфильтруем содержимое при помощи библиотеки nltk, для этого загрузим библиотеку и стоп-слова из английского языка

In [12]:
nltk.download('stopwords') 

stop_words = set(nltk_stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создадим корпус из лемматизированных и очищенных слов

In [16]:
corpus_lemm_train = X_train['lemm_text'].values
corpus_lemm_train.shape



(143362,)

Применим модель TfidfVectorizer и обучим её на наших тренировочных данных

In [17]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words,analyzer='word') 
tf_idf = count_tf_idf.fit_transform(corpus_lemm_train) 
tf_idf.shape

(143362, 142905)

Создадим так же корпус слов и для тестовой выборки

In [18]:
corpus_lemm_test = X_test['lemm_text'] #.values.astype()
corpus_lemm_test.shape

(15930,)

Трансформируем тестовую выборку

In [19]:
tf_test = count_tf_idf.transform(corpus_lemm_test)
tf_test.shape

(15930, 142905)

## Обучение

### LogisticRegression

In [30]:
%%time

# попытка задать веса классов вручную
w = [{0:89, 1:11}, {0:90, 1:10}]

clf_lr = LogisticRegression(random_state=12345, solver='sag')
parametrs_lr = { 'C': [14],           #range (5, 15),
              'max_iter': [12],       #range (10,15,2),
                'class_weight': ['balanced'] 
              }
grid_lr = GridSearchCV(clf_lr, parametrs_lr, cv=3, refit=True, scoring='f1')
grid_lr.fit(tf_idf, y_train)
grid_lr.best_params_

Audio(wave, rate=10000, autoplay=True)



# grid_lr.best_estimator_
# LogisticRegression(C=14, class_weight='balanced', max_iter=12,
#                    random_state=12345, solver='sag')




/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


CPU times: user 4.65 s, sys: 10.3 ms, total: 4.66 s
Wall time: 4.67 s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [31]:
results = pd.DataFrame(grid_lr.cv_results_)
display(results)
display(results[results['rank_test_score'] == 1])
fit_time = results[results['rank_test_score'] == 1]['mean_fit_time'].values[0]
predict_time = results[results['rank_test_score'] == 1]['mean_score_time'].values[0]
print(f'Fit time: {fit_time}, predict time:{predict_time}')

grid_lr.best_score_


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.037726,0.051223,0.01593,0.000342,14,balanced,12,"{'C': 14, 'class_weight': 'balanced', 'max_ite...",0.741614,0.743414,0.74451,0.743179,0.001194,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.037726,0.051223,0.01593,0.000342,14,balanced,12,"{'C': 14, 'class_weight': 'balanced', 'max_ite...",0.741614,0.743414,0.74451,0.743179,0.001194,1


Fit time: 1.037725607554118, predict time:0.0159304141998291


0.743179278537934

In [32]:
grid_lr.best_estimator_

LogisticRegression(C=14, class_weight='balanced', max_iter=12,
                   random_state=12345, solver='sag')

### CatBoostClassifier

In [ ]:
%%time

model_cat = CatBoostClassifier(loss_function= 'Logloss', 
                               iterations=20, #300
                               eval_metric='F1',
                               random_state=12345)

parameters_cat = {'depth': [10]} #

catboost_grid = model_cat.grid_search(parameters_cat, tf_idf, y_train,
                                      cv=3,
                                      verbose=True,
                                      refit=True
                                     )

Audio(wave, rate=10000, autoplay=True)



In [ ]:
catboost_grid["params"]
# {'depth': 10, 'iterations': 20}

In [ ]:
results = pd.DataFrame(catboost_grid['cv_results'])
results.sort_values(by='train-F1-mean', ascending=True)[0:1]
model_cat.best_score_

### SGDClassifier

In [33]:
%%time

#зададим веса
w = ['balanced'] #[{0:89, 1:11}, {0:90, 1:10}]

clf_sgdc = SGDClassifier(shuffle=True, random_state=12345)
parametrs_sgdc = {'alpha' : [1e-05],                 #np.linspace(0.00001, 0.0001, 15),
                  'learning_rate': ['optimal'],
                  'eta0' : [1e-05],                  #np.linspace(0.00001, 0.0001, 15),
                  'max_iter' : [9],                   #np.arange(5,10),
                  'class_weight': w 
                  }
grid_sgdc = GridSearchCV(clf_sgdc, parametrs_sgdc, cv=3, refit=True, scoring='f1')
grid_sgdc.fit(tf_idf, y_train)
grid_sgdc.best_params_

Audio(wave, rate=10000, autoplay=True)

# grid_sgdc.best_estimator_
# SGDClassifier(alpha=1e-05, eta0=1e-05, max_iter=9, random_state=12345)





/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


CPU times: user 1.45 s, sys: 286 ms, total: 1.74 s
Wall time: 1.76 s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


In [34]:
results = pd.DataFrame(grid_sgdc.cv_results_)
display(results)
display(results[results['rank_test_score'] == 1])
fit_time = results[results['rank_test_score'] == 1]['mean_fit_time'].values[0]
predict_time = results[results['rank_test_score'] == 1]['mean_score_time'].values[0]
print(f'Fit time: {fit_time}, predict time:{predict_time}')

grid_sgdc.best_score_


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_weight,param_eta0,param_learning_rate,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.391882,0.010353,0.016765,0.00064,0.00001,balanced,0.00001,optimal,9,"{'alpha': 1e-05, 'class_weight': 'balanced', '...",0.720735,0.733938,0.729851,0.728175,0.005519,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_weight,param_eta0,param_learning_rate,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.391882,0.010353,0.016765,0.00064,0.00001,balanced,0.00001,optimal,9,"{'alpha': 1e-05, 'class_weight': 'balanced', '...",0.720735,0.733938,0.729851,0.728175,0.005519,1


Fit time: 0.3918822606404622, predict time:0.01676495869954427


0.7281747818413457

In [35]:
grid_sgdc.best_estimator_

SGDClassifier(alpha=1e-05, class_weight='balanced', eta0=1e-05, max_iter=9,
              random_state=12345)

In [ ]:
# model_cat.best_score_
# {'learn': {'Logloss': 0.3012414101066201, 'F1': 0.8695380149156613}}

In [36]:
score = {'Модель':['LogisticRegression', 'SGDClassifier', 'Catboost'],  'Best score':[grid_lr.best_score_, grid_sgdc.best_score_, 0.869]} 

df_score = pd.DataFrame(score) 

display(df_score)

,Модель,Best score
0,LogisticRegression,0.743179
1,SGDClassifier,0.728175
2,Catboost,0.869000


Лучшей моделью является Логистическая Регрессия. Протестируем ее.

In [37]:
predicted_lr= grid_lr.predict(tf_test)

f1_log_r_1 = f1_score(y_test, predicted_lr)
print('Метрика F1 качества предсказания модели "LogisticRegression" равна :',f1_log_r_1)

Метрика F1 качества предсказания модели "LogisticRegression" равна : 0.7541620421753609


## Выводы

* Подходящая модель для магазина была найдена, это  **LogisticRegression**.
* Заданные параметы метрики качества достигнуты.